# GSMLS

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import json
import math
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from IPython.core.display import display, HTML
from concurrent.futures import ThreadPoolExecutor

pd.set_option('display.max_rows', None)

def preview_gsmls_df(mydf, save=False):
    columns = [
        'address', 'city', 'county', 'bedrooms', 'baths_full', 'baths_part',
        'sqft', 'price', 'tax', 'style', 'rooms', 'park_and_ride_name',
        'park_and_ride_duration_text', 'park_and_ride_type', 'park_and_ride_distance',
        'park_and_ride_bus_name', 'park_and_ride_bus_distance',
        'barnabas_duration_text', 'nyc_duration_text', 'lat', 'lng',]
    if save:
        mydf[columns].to_csv('gsmls.csv')
    display(HTML(
        """<style>.dataframe td {white-space: nowrap;}</style>""" +
        mydf[columns].to_html(
            formatters={
                '__index__':
                    lambda id: f'<a target="_blank" href="https://gsmls.herokuapp.com/properties/{id}">{id}</a>'},
                        escape=False)))

**Import global settings**

In [3]:
from settings import (blacklist_cities, counties, ONE_HOUR, HALF_HOUR, TEN_MINUTES,
                      MIN_PRICE, MAX_PRICE, MAX_TAX, MIN_BEDS, MIN_BATHS,)

print(MIN_PRICE, MAX_PRICE, MAX_TAX, MIN_BEDS, MIN_BATHS,)

300000 525000 16000 3 2


**Download listings from [GSMLS](https://www.gsmls.com/)**

In [4]:
from gsmls import get_listings

listings = []
for county in tqdm(counties):

    listings += get_listings(county,
                           min_list_price=MIN_PRICE,
                           max_list_price=int(math.floor((MAX_PRICE+MIN_PRICE)/2)),
                           min_bedrooms=MIN_BEDS,
                           min_bathrooms=MIN_BATHS)

    listings += get_listings(county,
                           min_list_price=int(math.ceil((MAX_PRICE+MIN_PRICE)/2)),
                           max_list_price=MAX_PRICE,
                           min_bedrooms=MIN_BEDS,
                           min_bathrooms=MIN_BATHS)

assert len(set([x['id'] for x in listings])) == len(listings)

print(f"Downloaded {len(listings)} listings from GSMLS.")
listings = [x for x in listings if x['address']]
print(f"Filtered down to {len(listings)} listings due to null address.")

HBox(children=(IntProgress(value=0, max=8), HTML(value='')))


Downloaded 1599 listings from GSMLS.
Filtered down to 1521 listings due to null address.


**Geocode each address using Google Maps API**

In [5]:
from geolocate import add_geocode_to_listing
for listing in tqdm(listings):
    if not listing.get('address'): continue
    add_geocode_to_listing(listing)

HBox(children=(IntProgress(value=0, max=1521), HTML(value='')))

could not find county for 3426835 46 Veranda Ave, North Caldwell, NJ, USA [{'long_name': '46', 'short_name': '46', 'types': ['street_number']}, {'long_name': 'Veranda Avenue', 'short_name': 'Veranda Ave', 'types': ['route']}, {'long_name': 'North Caldwell', 'short_name': 'North Caldwell', 'types': ['locality', 'political']}, {'long_name': 'New Jersey', 'short_name': 'NJ', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}]
could not geocode: 4 Shadtree Lane Mount Arlington Boro, NJ or 4 Shadtree Lane Mount Arlington, NJ
could not geocode: 2 Shadetree Lane Mount Arlington Boro, NJ or 2 Shadetree Lane Mount Arlington, NJ
could not geocode: 15 Shadetree Ln Mount Arlington Boro, NJ or 15 Shadetree Ln Mount Arlington, NJ
could not geocode: 5 Harbor Front Plz A5 Elizabeth City, NJ or 5 Harbor Front Plz A5 Elizabeth, NJ
could not geocode: 6 Harbor Front Ct Elizabeth City, NJ or 6 Harbor Front Ct Elizabe

**Find closest NJ Transit Park and Ride to each address**

In [6]:
from commute import add_closest_park_and_ride_to_listing

for i in tqdm(range(len(listings))):
    listing = listings[i]
    add_closest_park_and_ride_to_listing(listing)

HBox(children=(IntProgress(value=0, max=1521), HTML(value='')))

**Get commute time for each address through both park and ride and walking**

In [7]:
from commute import add_commute_to_listing

for i in tqdm(range(len(listings))):
    listing = listings[i]
    add_commute_to_listing(listing)

print(sum(1 for listing in listings if listing.get('park_and_ride_duration')),
      "out of", len(listings), "have park and ride commute times.")

print(sum(1 for listing in listings if listing.get('nyc_duration')),
  "out of", len(listings), "have walking commute times.")

HBox(children=(IntProgress(value=0, max=1521), HTML(value='')))


1502 out of 1521 have park and ride commute times.
1112 out of 1521 have walking commute times.


In [8]:
gsmls_filters = lambda x: ((x.address.notnull()) &
                            (~x.city.isin(blacklist_cities)) &
                            (x.barnabas_duration < HALF_HOUR + TEN_MINUTES) &                           
                            ((x.park_and_ride_duration1 < TEN_MINUTES) & (x.park_and_ride_duration < ONE_HOUR + TEN_MINUTES) | (x.nyc_duration < ONE_HOUR + TEN_MINUTES)) &
                            (~x['style'].str.contains('Townhouse', case=False)) &
                            # (~x['style'].str.contains('Bi-Level', case=False)) &
                            (~x['style'].str.contains('Cape Cod', case=False)) &
                            (~x['style'].str.contains('Multi Floor Unit', case=False)) &
                            (~x['style'].str.contains('1/2 Duplex', case=False)) &
                            (~x.heat_source.str.contains('oil', case=False, na=True)) &
                            (~x.heat_system.str.contains('radiator', case=False, na=True)) &
                            # (x.cool_system.str.contains('central', case=False, na=True)) &
                            (~x.water.str.contains('well', case=False, na=True)) &
                            (~x.sewer.str.contains('septic', case=False, na=True))
                          )

def show_gsmls():
    df = pd.DataFrame(listings)
    df.set_index('id', inplace=True)
    df = df[gsmls_filters(df)]
    df = df.sort_values(by=['park_and_ride_duration', 'park_and_ride_distance'], ascending=[True, True])
    return df

In [9]:
df = show_gsmls()
df = df[(df.tax < MAX_TAX)]
df = df.sort_values('city')
print(len(df), "listings after filtering")
preview_gsmls_df(df)

235 listings after filtering


,address,city,county,bedrooms,baths_full,baths_part,sqft,price,tax,style,rooms,park_and_ride_name,park_and_ride_duration_text,park_and_ride_type,park_and_ride_distance,park_and_ride_bus_name,park_and_ride_bus_distance,barnabas_duration_text,nyc_duration_text,lat,lng
id,,,,,,,,,,,,,,,,,,,,,
3428137,70 W 14th St,Bayonne,Hudson,3,2,1,1460.0,379000,10488,Colonial,12,8th Street (Bayonne),"3 mins, 1 hour 2 mins",light_rail,0.339859,Newark Penn Station Bus Lanes,5.537004,40 mins,1 hour 5 mins,40.658909,-74.126869
3433870,39 Greenbriar,Berkeley Heights,Union,4,2,1,NaN,515000,10412,Bi-Level,8,Berkeley Heights,"5 mins, 1 hour 12 mins",rail,0.699960,Watchung Park & Ride,2.978442,25 mins,1 hour 5 mins,40.685074,-74.429815
3400950,47 Hearthstone Rd,Bloomfield,Essex,4,2,0,NaN,350000,12699,Split Level,10,Watchung Avenue,"6 mins, 45 mins",rail,1.459975,Allwood Road,1.638773,25 mins,57 mins,40.830263,-74.179093
3434778,79 Winding Ln,Bloomfield,Essex,4,3,0,3054.0,489000,13683,Split Level,11,Watchung Avenue,"6 mins, 45 mins",rail,1.334357,Allwood Road,1.641766,24 mins,56 mins,40.830521,-74.181507
3426760,102 Beverly Rd,Bloomfield,Essex,3,2,0,NaN,304000,11438,Split Level,7,Watchung Avenue,"6 mins, 45 mins",rail,1.406556,Allwood Road,1.953135,23 mins,53 mins,40.825814,-74.180543
3425147,85 Bellevue Ter,Bloomfield,Essex,3,2,1,NaN,424900,12668,Split Level,8,Allwood Road,"4 mins, 33 mins",bus,1.063173,Allwood Road,1.063173,27 mins,58 mins,40.840759,-74.185958
3433166,175 Jerome Pl,Bloomfield,Essex,4,3,0,1650.0,459000,8715,Colonial,8,Bloomfield,"5 mins, 36 mins",rail,0.632988,Clifton Commons,3.454818,21 mins,1 hour 12 mins,40.791904,-74.188019
3419648,13 BELLEVUE AVE,Bloomfield,Essex,3,3,0,NaN,425000,14104,Colonial,7,Watchung Avenue,"5 mins, 45 mins",rail,1.211963,Allwood Road,1.266570,26 mins,52 mins,40.837542,-74.186360
3426528,128 Willow St,Bloomfield,Essex,3,2,1,NaN,349000,10124,Colonial,7,Watsessing Avenue,"3 mins, 33 mins",rail,0.293689,Newark Penn Station Bus Lanes,4.057832,21 mins,54 mins,40.784442,-74.203585


In [10]:
import folium

home = (40.8961863, -74.1726829,)

m = folium.Map(location=home, zoom_start=12)

folium.Marker(home, tooltip=f'<i>115 Dumont Ave. Clifton, NJ</i>', icon=folium.Icon(color='green')).add_to(m)

for id, row in df.iterrows():
    url = f"https://gsmls.herokuapp.com/properties/{id}"
    address = row.address + ", " + row.city
    popup = f"""
    Address: {address}<br/>
    MLS: <a target="_blank" href="{url}">{id}</a><br/>
    Price: {row.price}<br/>
    Tax: {row.tax}<br/>
    Dad (Park and Ride): {row.park_and_ride_duration_text}<br/>
    Dad (Walk and Ride): {row.nyc_duration_text}<br/>
    Mom: {row.barnabas_duration_text}
    """
    folium.Marker((row.lat, row.lng,), popup=f'<i>{popup}</i>', tooltip=f'<i>{address}</i>').add_to(m)

m